In [117]:
import pandas as pd
import numpy as np
from time import time
from config_loader import load
import argparse
import sys
import seaborn as sns
from MyDataUnderstanding import featureAnalysis
from MyPreprocessing import MyPreprocessing
import numpy as np
from time import time
import matplotlib.pyplot as plt
from model.models import models_perform
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from statistics import mean
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from detect_outliers import detect_outliers

In [118]:
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.svm import SVC
from xgboost import XGBClassifier as XGB
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from model.MyIBL import MyIBL as IBL
from sklearn.linear_model import LogisticRegression as LG

In [22]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 500)

In [23]:
def getData(path, filenames_type):
    '''
    features_lst = [
        "Pclass", "Survived", "Name", "Sex", "Age",
        "Sibsp", "Parch", "Ticket", "Fare", "Cabin","Embarked",
        "Boat","Body","Home.dest"]
    '''
    if filenames_type == 'train':
        filename = 'train'
    elif filenames_type == 'test':
        filename = 'test'
    else:
        filename = 'titanicAll'
        
    df_features = pd.read_csv(path + filename + '.csv',
                           sep=',')

    if filename_type not in ['train', 'test']:
        # drop unnecessary columns that don't exist in the official data
        df_features.drop(['Boat', 'Body', 'Home.dest'],
                           axis=1,
                           inplace=True)
    #labels = df_features['Survived']
    #df_features = df_features.drop(['Survived'], axis=1)
    return df_features

In [24]:
parser = argparse.ArgumentParser()
parser.add_argument(
    "-c", "--config", default="titanic.cfg",
    help="specify the location of the clustering config file"
)
args, _ = parser.parse_known_args()

config_file = args.config
config = load(config_file)

##
verbose = config.get('titanic', 'verbose')
path = config.get('titanic', 'path') + '/'
file_type = config.get('titanic', 'file_type')

In [25]:
print(file_type)
filename_type = 'train'
if file_type == 'all':
    filename_type = 'other'

trainData = getData(path, filename_type)

# Preprocessing
trainPreprocess = MyPreprocessing(filename_type)

## test
filename_type = 'test'
testData = getData(path, filename_type)
# Preprocessing
testPreprocess = MyPreprocessing(filename_type)

traintest


In [5]:
trainData = getData(path, filename_type)

# Preprocessing
trainPreprocess = MyPreprocessing(filename_type)

## test
filename_type = 'test'
testData = getData(path, filename_type)
# Preprocessing
testPreprocess = MyPreprocessing(filename_type)

In [26]:
trainPreprocess.fit(trainData)
df_train = trainPreprocess.new_df
# the labels "Survived"
labels = trainPreprocess.labels_

testPreprocess.fit(testData)
df_test = testPreprocess.new_df
labels_test = testPreprocess.labels_

# fix missing columns because of NaNs and one hot encoding without dummy_na
if df_train.shape[1] != df_test.shape[1]:
    missing_cols = set(df_test.columns) - set(df_train.columns)
    for col in missing_cols:
        #df_train[col] = np.zeros([df_train.shape[0], 1])
        df_test.drop([col], axis=1, inplace=True)

    missing_cols = set(df_train.columns) - set(df_test.columns)
    for col in missing_cols:
        #df_test[col] = np.zeros([df_test.shape[0], 1])
        df_train.drop([col], axis=1, inplace=True)

In [27]:
df_train.columns

Index(['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Title_Mr0', 'Title_Mr50', 'Title_Mrs', 'Title_Ms', 'Age_bin_Kid', 'Age_bin_Teenager', 'Age_bin_Adult', 'FamilySize', 'Family_bin_SmallFamily', 'Family_bin_BigFamily', 'Family_bin_Team', 'Fare_bin_Median', 'Fare_bin_Average', 'Fare_bin_High', 'Em_C', 'Em_Q', 'Em_S', 'Cabin_A', 'Cabin_B', 'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_X', 'Mother', 'Father', 'Daughter', 'Son', 'Orphan', 'RichWoman', 'MiddleClassWoman', 'PoorWoman', 'RichMan', 'MiddleClassMan', 'PoorMan', 'RichGirl', 'MiddleClassGirl', 'PoorGirl', 'RichBoy', 'MiddleClassBoy', 'PoorBoy', 'Pclass_2', 'Pclass_3'], dtype='object')

In [8]:
df_test.columns

Index(['Sex', 'Age', 'SibSp', 'Parch', 'Title_Mr0', 'Title_Mr50', 'Title_Mrs', 'Title_Ms', 'Age_bin_Kid', 'Age_bin_Teenager', 'Age_bin_Adult', 'FamilySize', 'Family_bin_SmallFamily', 'Family_bin_BigFamily', 'Family_bin_Team', 'Fare_bin_Median', 'Fare_bin_Average', 'Fare_bin_High', 'Em_C', 'Em_Q', 'Em_S', 'Cabin_A', 'Cabin_B', 'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_X', 'Mother', 'Father', 'Daughter', 'Son', 'Orphan', 'RichWoman', 'MiddleClassWoman', 'PoorWoman', 'RichMan', 'MiddleClassMan', 'PoorMan', 'RichGirl', 'MiddleClassGirl', 'PoorGirl', 'RichBoy', 'MiddleClassBoy', 'PoorBoy', 'Pclass_2', 'Pclass_3'], dtype='object')

In [9]:
#df_test.dtypes

In [28]:
def feature_importance(clf, df_train):
    if hasattr(clf, 'feature_importances_'):
        name = str(clf.__class__).split('.')[-1][:-2]
        feat_imp = pd.DataFrame({'importance':clf.feature_importances_})    
        feat_imp['feature'] = df_train.columns
        feat_imp.sort_values(by='importance', ascending=False, inplace=True)
        #feat_imp = feat_imp.iloc[:top_n]
        feat_imp.sort_values(by='importance', inplace=True)
        feat_imp = feat_imp.set_index('feature', drop=True)
        feat_imp.plot.barh(title='Feature Importance', figsize=(10,10))
        plt.xlabel('%s Feature Importance Score' %name)
        plt.show()
        return feat_imp
    return pd.DataFrame()

In [29]:
def ensemble(clf, x_train, y_train, x_test):
    
    name = str(clf.__class__).split('.')[-1][:-2]
    kf = KFold(n_splits=5)

    ntrain = x_train.shape[0]
    ntest = x_test.shape[0]

    #ens_train = np.zeros((ntrain,))
    ens_train = np.array([])
    #ens_test = np.zeros((ntest,))
    #test_kf = np.empty((kf.n_folds, ntest))
    test_kf = np.zeros((kf.n_splits, ntest))
        
    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.loc[train_index]
        y_tr = y_train.loc[train_index]
        x_te = x_train.loc[test_index]

        clf.fit(x_tr, y_tr)

        #ens_train[test_index] = clf.predict(x_te)
        ens_train = np.concatenate([ens_train, clf.predict(x_te)])
        # predict from the whole train set 
        #clf.fit(x_train,y_train)
        test_kf[i, :] = clf.predict(x_test)
    
    # features for next ensemble layer
    ens_train = pd.DataFrame(ens_train.reshape(-1,1), columns=[name])
    
    # combine for test
    ens_test = pd.DataFrame(test_kf.T).mode(axis=1)
    ens_test.columns = [name]
    
    return ens_train, ens_test

In [45]:
def modeler(clf, data, labels, train_fidx, validation_fidx, name_prefix):
    name = name_prefix + str(clf.__class__).split('.')[-1][:-2]
    #print(name)
    folds_accuracy = list()
    duration = list()
    start = time()
    for idx, trf in enumerate(train_fidx):
        clf.fit(data.loc[trf], labels.loc[trf])
        prediction_labels = clf.predict(data.loc[validation_fidx[idx]])
        folds_accuracy.append(accuracy_score(labels.loc[validation_fidx[idx]], prediction_labels))
        
    mean_acc = mean(folds_accuracy)
    duration = time() - start
    #print(name, mean_acc, "accuracy at validation stage,", duration, 's')
    df_folds_accuracy = pd.DataFrame([folds_accuracy], index=[name])
    df_folds = pd.DataFrame([[mean_acc, duration/len(train_fidx)]], columns=['Accuracy', 'Duration'], index=[name])
    return clf.fit(data, labels), df_folds, df_folds_accuracy

In [31]:
def get_cv_data(df_train, cv=5):
    kf = KFold(n_splits=cv)
    folds = [(train_idx, validation_idx) for train_idx, validation_idx in kf.split(df_train)]
    train_idx = [f[0] for f in folds]
    validation_idx = [f[1] for f in folds]
    return train_idx, validation_idx

In [47]:
def run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True):
    models = []
    df_ens_features = pd.DataFrame()
    df_ens_test = pd.DataFrame()
    df_model_cvaccuracies = pd.DataFrame()
    df_model_info = pd.DataFrame()
    for clf in clfs:
        name = str(clf.__class__).split('.')[-1][:-2]
        
        # cross validation
        if cv:
            model, df_folds, df_folds_accuracy = modeler(clf, df_train, labels, train_idx, validation_idx, name_prefix)
            models.append(model)
            df_model_cvaccuracies = pd.concat([df_model_cvaccuracies, df_folds_accuracy], sort=False)
            df_model_info = pd.concat([df_model_info, df_folds], sort=False)  
            
            #feature_importance(clf, df_train)
        
        # building ensemble
        if cv and ens:
            #df_feature = pd.DataFrame(cross_val_predict(clf, df_train, labels, cv=ens_kf).reshape(-1,1), columns=[name])
            df_ens_feature, ens_test = ensemble(clf, df_train, labels, df_test)
            df_ens_features = pd.concat([df_ens_features, df_ens_feature], axis=1, sort=False).reset_index(drop=True)
            df_ens_test = pd.concat([df_ens_test, ens_test], axis=1, sort=False).reset_index(drop=True)
            
    return models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test
        

In [63]:
clfs = [IBL(), RF(n_estimators =12), SVC(gamma='scale'), XGB(), MLP(max_iter=1000), KNN(), LDA()]
#clfs = [RF(n_estimators =12), SVC(gamma='scale'), XGB(), KNN(), LDA()]

In [33]:
train_idx, validation_idx = get_cv_data(df_train, cv=5)

In [37]:
import warnings
warnings.filterwarnings("ignore")

In [48]:
name_prefix = '';
models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)

In [49]:
df_model_cvaccuracies

,0,1,2,3,4
MyIBL,0.720670,0.764045,0.758427,0.713483,0.814607
RandomForestClassifier,0.776536,0.792135,0.820225,0.780899,0.825843
SVC,0.837989,0.825843,0.820225,0.797753,0.876404
XGBClassifier,0.826816,0.825843,0.853933,0.792135,0.876404
MLPClassifier,0.821229,0.769663,0.842697,0.803371,0.865169
KNeighborsClassifier,0.798883,0.797753,0.842697,0.803371,0.865169
KNeighborsClassifier,0.798883,0.797753,0.842697,0.803371,0.865169
LinearDiscriminantAnalysis,0.798883,0.814607,0.808989,0.797753,0.853933


In [50]:
df_model_info

,Accuracy,Duration
MyIBL,0.754246,2.589207
RandomForestClassifier,0.799127,0.024185
SVC,0.831643,0.031181
XGBClassifier,0.835026,0.141513
MLPClassifier,0.820426,4.076642
KNeighborsClassifier,0.821574,0.013392
KNeighborsClassifier,0.821574,0.013791
LinearDiscriminantAnalysis,0.814833,0.018404


In [51]:
print('Original')
print('##################################')
name_prefix = 'ALL_' + str(len(df_train.columns) + '_');
models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)
df_model_info

Original
##################################


,Accuracy,Duration
ALL_49MyIBL,0.744134,2.617616
ALL_49RandomForestClassifier,0.791275,0.024985
ALL_49SVC,0.831643,0.032380
ALL_49XGBClassifier,0.835026,0.144911
ALL_49MLPClassifier,0.822673,4.181963
ALL_49KNeighborsClassifier,0.821574,0.014391
ALL_49KNeighborsClassifier,0.821574,0.014791
ALL_49LinearDiscriminantAnalysis,0.814833,0.018779


In [61]:
print('PCA')
print('##################################')
df_results_pca = pd.DataFrame()
df_results_pca_cv = pd.DataFrame()
for n_dim in range(15, 22):
    print(n_dim, ' dimensions')
    pca_train, pca_test, ev = MyFeatureSelection.applyPCA(df_train, df_test, n_dim)
    name_prefix = 'PCA_d_' + str(n_dim) + '/' + str(len(df_train.columns)) + '_ev_' + str(round(ev,3)) + '_'
    models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)
    df_results_pca_cv = pd.concat([df_results_pca_cv, df_model_cvaccuracies], axis=0)
    df_results_pca = pd.concat([df_results_pca, df_model_info], axis=0)

PCA
##################################

15  dimensions:

16  dimensions:

17  dimensions:

18  dimensions:

19  dimensions:

20  dimensions:

21  dimensions:


In [80]:
df_results_pca

,Accuracy,Duration
PCA_d_15/49_ev_0.888_MyIBL,0.731762,2.581020
PCA_d_15/49_ev_0.888_RandomForestClassifier,0.797985,0.025305
PCA_d_15/49_ev_0.888_SVC,0.831643,0.031710
PCA_d_15/49_ev_0.888_XGBClassifier,0.835026,0.144509
PCA_d_15/49_ev_0.888_MLPClassifier,0.822679,4.139202
PCA_d_15/49_ev_0.888_KNeighborsClassifier,0.821574,0.013592
PCA_d_15/49_ev_0.888_KNeighborsClassifier,0.821574,0.013996
PCA_d_15/49_ev_0.888_LinearDiscriminantAnalysis,0.814833,0.017821
PCA_d_16/49_ev_0.901_MyIBL,0.720601,2.662204
PCA_d_16/49_ev_0.901_RandomForestClassifier,0.796887,0.024785


In [120]:
df_results_pca_order = df_results_pca.copy()
df_results_pca_order.sort_values('Accuracy')

,Accuracy,Duration
PCA_d_16/49_ev_0.901_MyIBL,0.720601,2.662204
PCA_d_21/49_ev_0.949_MyIBL,0.725089,2.619470
PCA_d_15/49_ev_0.888_MyIBL,0.731762,2.581020
PCA_d_20/49_ev_0.942_MyIBL,0.736338,2.507238
PCA_d_17/49_ev_0.913_MyIBL,0.737386,2.576706
PCA_d_18/49_ev_0.924_MyIBL,0.738566,2.569580
PCA_d_19/49_ev_0.933_MyIBL,0.764334,2.636394
PCA_d_16/49_ev_0.901_RandomForestClassifier,0.796887,0.024785
PCA_d_15/49_ev_0.888_RandomForestClassifier,0.797985,0.025305
PCA_d_19/49_ev_0.933_RandomForestClassifier,0.798004,0.024315


In [79]:
df_results_pca_cv

,0,1,2,3,4
PCA_d_15/49_ev_0.888_MyIBL,0.731844,0.747191,0.702247,0.752809,0.724719
PCA_d_15/49_ev_0.888_RandomForestClassifier,0.793296,0.769663,0.837079,0.786517,0.803371
PCA_d_15/49_ev_0.888_SVC,0.837989,0.825843,0.820225,0.797753,0.876404
PCA_d_15/49_ev_0.888_XGBClassifier,0.826816,0.825843,0.853933,0.792135,0.876404
PCA_d_15/49_ev_0.888_MLPClassifier,0.815642,0.769663,0.853933,0.808989,0.865169
PCA_d_15/49_ev_0.888_KNeighborsClassifier,0.798883,0.797753,0.842697,0.803371,0.865169
PCA_d_15/49_ev_0.888_KNeighborsClassifier,0.798883,0.797753,0.842697,0.803371,0.865169
PCA_d_15/49_ev_0.888_LinearDiscriminantAnalysis,0.798883,0.814607,0.808989,0.797753,0.853933
PCA_d_16/49_ev_0.901_MyIBL,0.664804,0.747191,0.668539,0.764045,0.758427
PCA_d_16/49_ev_0.901_RandomForestClassifier,0.770950,0.780899,0.820225,0.769663,0.842697


In [70]:
max(df_results_pca['Accuracy'])

0.8350260498399348

In [77]:
print('ICA')
print('##################################')
df_results_ica = pd.DataFrame()
df_results_ica_cv = pd.DataFrame()
for n_dim in [25,30,35,40,45]:
    print(n_dim, ' dimensions')
    ica_train, ica_test = MyFeatureSelection.applyICA(df_train, df_test, n_dim)
    name_prefix = 'ICA_d_' + str(n_dim) + '/' + str(len(df_train.columns))
    models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)
    df_results_ica_cv = pd.concat([df_results_ica_cv, df_model_cvaccuracies], axis=0)
    df_results_ica = pd.concat([df_results_ica, df_model_info], axis=0)

ICA
##################################
25  dimensions
30  dimensions
35  dimensions
40  dimensions
45  dimensions


In [81]:
df_results_ica

,Accuracy,Duration
ICA_d_25/49MyIBL,0.744122,2.591335
ICA_d_25/49RandomForestClassifier,0.806942,0.024985
ICA_d_25/49SVC,0.831643,0.032180
ICA_d_25/49XGBClassifier,0.835026,0.149387
ICA_d_25/49MLPClassifier,0.812567,4.687885
ICA_d_25/49KNeighborsClassifier,0.821574,0.014191
ICA_d_25/49LinearDiscriminantAnalysis,0.814833,0.018078
ICA_d_30/49MyIBL,0.764346,2.637952
ICA_d_30/49RandomForestClassifier,0.806955,0.024899
ICA_d_30/49SVC,0.831643,0.031778


In [82]:
df_results_ica_cv

,0,1,2,3,4
ICA_d_25/49MyIBL,0.731844,0.741573,0.713483,0.719101,0.814607
ICA_d_25/49RandomForestClassifier,0.821229,0.792135,0.803371,0.808989,0.808989
ICA_d_25/49SVC,0.837989,0.825843,0.820225,0.797753,0.876404
ICA_d_25/49XGBClassifier,0.826816,0.825843,0.853933,0.792135,0.876404
ICA_d_25/49MLPClassifier,0.815642,0.769663,0.831461,0.797753,0.848315
ICA_d_25/49KNeighborsClassifier,0.798883,0.797753,0.842697,0.803371,0.865169
ICA_d_25/49LinearDiscriminantAnalysis,0.798883,0.814607,0.808989,0.797753,0.853933
ICA_d_30/49MyIBL,0.731844,0.797753,0.758427,0.769663,0.764045
ICA_d_30/49RandomForestClassifier,0.810056,0.775281,0.842697,0.769663,0.837079
ICA_d_30/49SVC,0.837989,0.825843,0.820225,0.797753,0.876404


In [121]:
df_results_ica_order = df_results_ica.copy()
df_results_ica_order.sort_values('Accuracy')

,Accuracy,Duration
ICA_d_35/49MyIBL,0.726131,2.557562
ICA_d_45/49MyIBL,0.728372,2.831865
ICA_d_25/49MyIBL,0.744122,2.591335
ICA_d_40/49MyIBL,0.744122,2.579561
ICA_d_30/49MyIBL,0.764346,2.637952
ICA_d_35/49RandomForestClassifier,0.792417,0.024434
ICA_d_45/49RandomForestClassifier,0.796899,0.024385
ICA_d_40/49RandomForestClassifier,0.803584,0.024806
ICA_d_25/49RandomForestClassifier,0.806942,0.024985
ICA_d_30/49RandomForestClassifier,0.806955,0.024899


In [83]:
max(df_results_ica['Accuracy'])

0.8350260498399348

In [84]:
print('INFO GAIN SELECTION')
print('##################################')
df_results_ig = pd.DataFrame()
df_results_ig_cv = pd.DataFrame()
for n_dim in [25,30,35,40,45]:
    print(n_dim, ' dimensions')
    ig_train, ig_test = MyFeatureSelection.InfoGainSelection(df_train, df_test, labels, n_dim)
    name_prefix = 'IG_d_' + str(n_dim) + '/' + str(len(df_train.columns))
    models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)
    df_results_ig_cv = pd.concat([df_results_ig_cv, df_model_cvaccuracies], axis=0)
    df_results_ig = pd.concat([df_results_ig, df_model_info], axis=0)

INFO GAIN SELECTION
##################################
25  dimensions
30  dimensions
35  dimensions
40  dimensions
45  dimensions


In [85]:
df_results_ig

,Accuracy,Duration
IG_d_25/49MyIBL,0.753154,2.556570
IG_d_25/49RandomForestClassifier,0.799115,0.024838
IG_d_25/49SVC,0.831643,0.031780
IG_d_25/49XGBClassifier,0.835026,0.141282
IG_d_25/49MLPClassifier,0.812579,4.604124
IG_d_25/49KNeighborsClassifier,0.821574,0.014193
IG_d_25/49LinearDiscriminantAnalysis,0.814833,0.017663
IG_d_30/49MyIBL,0.732911,2.543963
IG_d_30/49RandomForestClassifier,0.800239,0.023802
IG_d_30/49SVC,0.831643,0.031593


In [122]:
df_results_ig_order = df_results_ig.copy()
df_results_ig_order.sort_values('Accuracy')

,Accuracy,Duration
IG_d_30/49MyIBL,0.732911,2.543963
IG_d_35/49MyIBL,0.737380,2.578618
IG_d_45/49MyIBL,0.747473,2.678083
IG_d_25/49MyIBL,0.753154,2.556570
IG_d_40/49MyIBL,0.756531,2.618336
IG_d_40/49RandomForestClassifier,0.786780,0.024588
IG_d_25/49RandomForestClassifier,0.799115,0.024838
IG_d_30/49RandomForestClassifier,0.800239,0.023802
IG_d_45/49RandomForestClassifier,0.801343,0.024202
IG_d_35/49RandomForestClassifier,0.805838,0.024214


In [86]:
df_results_ig_cv

,0,1,2,3,4
IG_d_25/49MyIBL,0.692737,0.764045,0.764045,0.752809,0.792135
IG_d_25/49RandomForestClassifier,0.787709,0.792135,0.820225,0.752809,0.842697
IG_d_25/49SVC,0.837989,0.825843,0.820225,0.797753,0.876404
IG_d_25/49XGBClassifier,0.826816,0.825843,0.853933,0.792135,0.876404
IG_d_25/49MLPClassifier,0.804469,0.775281,0.825843,0.808989,0.848315
IG_d_25/49KNeighborsClassifier,0.798883,0.797753,0.842697,0.803371,0.865169
IG_d_25/49LinearDiscriminantAnalysis,0.798883,0.814607,0.808989,0.797753,0.853933
IG_d_30/49MyIBL,0.709497,0.747191,0.769663,0.691011,0.747191
IG_d_30/49RandomForestClassifier,0.787709,0.797753,0.853933,0.758427,0.803371
IG_d_30/49SVC,0.837989,0.825843,0.820225,0.797753,0.876404


In [87]:
max(df_results_ig['Accuracy'])

0.8350260498399348

In [88]:
print('ANOVA SELECTION')
print('##################################')
df_results_an = pd.DataFrame()
df_results_an_cv = pd.DataFrame()
for n_dim in [25,30,35,40,45]:
    print(n_dim, ' dimensions')
    an_train, an_test = MyFeatureSelection.AnovaSelection(df_train, df_test, labels, n_dim)
    name_prefix = 'AN_d_' + str(n_dim) + '/' + str(len(df_train.columns))
    models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)
    df_results_an_cv = pd.concat([df_results_an_cv, df_model_cvaccuracies], axis=0)
    df_results_an = pd.concat([df_results_an, df_model_info], axis=0)

ANOVA SELECTION
##################################
25  dimensions
30  dimensions
35  dimensions
40  dimensions
45  dimensions


In [89]:
df_results_an

,Accuracy,Duration
AN_d_25/49MyIBL,0.738491,2.627603
AN_d_25/49RandomForestClassifier,0.806999,0.024590
AN_d_25/49SVC,0.831643,0.031388
AN_d_25/49XGBClassifier,0.835026,0.143726
AN_d_25/49MLPClassifier,0.822673,4.314940
AN_d_25/49KNeighborsClassifier,0.821574,0.013592
AN_d_25/49LinearDiscriminantAnalysis,0.814833,0.017985
AN_d_30/49MyIBL,0.750876,2.596134
AN_d_30/49RandomForestClassifier,0.801394,0.025185
AN_d_30/49SVC,0.831643,0.032397


In [123]:
df_results_an_order = df_results_ica.copy()
df_results_an_order.sort_values('Accuracy')

,Accuracy,Duration
ICA_d_35/49MyIBL,0.726131,2.557562
ICA_d_45/49MyIBL,0.728372,2.831865
ICA_d_25/49MyIBL,0.744122,2.591335
ICA_d_40/49MyIBL,0.744122,2.579561
ICA_d_30/49MyIBL,0.764346,2.637952
ICA_d_35/49RandomForestClassifier,0.792417,0.024434
ICA_d_45/49RandomForestClassifier,0.796899,0.024385
ICA_d_40/49RandomForestClassifier,0.803584,0.024806
ICA_d_25/49RandomForestClassifier,0.806942,0.024985
ICA_d_30/49RandomForestClassifier,0.806955,0.024899


In [90]:
df_results_an_cv

,0,1,2,3,4
AN_d_25/49MyIBL,0.743017,0.702247,0.730337,0.747191,0.769663
AN_d_25/49RandomForestClassifier,0.770950,0.808989,0.848315,0.792135,0.814607
AN_d_25/49SVC,0.837989,0.825843,0.820225,0.797753,0.876404
AN_d_25/49XGBClassifier,0.826816,0.825843,0.853933,0.792135,0.876404
AN_d_25/49MLPClassifier,0.821229,0.780899,0.853933,0.808989,0.848315
AN_d_25/49KNeighborsClassifier,0.798883,0.797753,0.842697,0.803371,0.865169
AN_d_25/49LinearDiscriminantAnalysis,0.798883,0.814607,0.808989,0.797753,0.853933
AN_d_30/49MyIBL,0.720670,0.741573,0.752809,0.747191,0.792135
AN_d_30/49RandomForestClassifier,0.759777,0.786517,0.825843,0.814607,0.820225
AN_d_30/49SVC,0.837989,0.825843,0.820225,0.797753,0.876404


In [91]:
max(df_results_ig['Accuracy'])

0.8350260498399348

In [102]:
# Random forest estimator for feature selection

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(df_train, labels)
#model = SelectFromModel(lsvc, prefit=True)
model = SelectFromModel(RF(n_estimators=100, random_state=101), threshold='median')
model.fit(df_train, labels)
train_new = model.transform(df_train)
feature_idx = model.get_support()
df_train_fs = pd.DataFrame(train_new, columns=df_train.columns[feature_idx])
test_new = model.transform(df_test)
df_test_fs = pd.DataFrame(test_new, columns=df_test.columns[feature_idx])


print('RANDOM FOREST SELECTION')
print('##################################')
rf_results_an = pd.DataFrame()
rf_results_an_cv = pd.DataFrame()
print(len(df_train_fs.columns), ' dimensions')
name_prefix = 'RF_d_' + str(len(df_train_fs.columns)) + '/' + str(len(df_train.columns))
models, rf_results_an_cv, rf_results_an, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)

RANDOM FOREST SELECTION
##################################
25  dimensions


In [94]:
rf_results_an

,Accuracy,Duration
RF_d_25/49MyIBL,0.748578,2.541520
RF_d_25/49RandomForestClassifier,0.800226,0.024202
RF_d_25/49SVC,0.831643,0.031503
RF_d_25/49XGBClassifier,0.835026,0.148768
RF_d_25/49MLPClassifier,0.821562,4.296625
RF_d_25/49KNeighborsClassifier,0.821574,0.013593
RF_d_25/49LinearDiscriminantAnalysis,0.814833,0.018652


In [95]:
rf_results_an_cv

,0,1,2,3,4
RF_d_25/49MyIBL,0.765363,0.735955,0.730337,0.724719,0.786517
RF_d_25/49RandomForestClassifier,0.798883,0.775281,0.803371,0.792135,0.831461
RF_d_25/49SVC,0.837989,0.825843,0.820225,0.797753,0.876404
RF_d_25/49XGBClassifier,0.826816,0.825843,0.853933,0.792135,0.876404
RF_d_25/49MLPClassifier,0.810056,0.775281,0.831461,0.825843,0.865169
RF_d_25/49KNeighborsClassifier,0.798883,0.797753,0.842697,0.803371,0.865169
RF_d_25/49LinearDiscriminantAnalysis,0.798883,0.814607,0.808989,0.797753,0.853933


In [119]:
print('Lasso Regression Selection')
print('##################################')
df_results_an = pd.DataFrame()
df_results_an_cv = pd.DataFrame()
for alpha in [0.05, 0.005]:
    df_train, df_test = MyFeatureSelection.LassoRegressionSelection(df_train, df_test, labels, alpha)
    print(len(df_train.columns), ' dimensions')
    name_prefix = 'LR_d_' + str(n_dim) + '/' + str(len(df_train.columns))
    models, df_model_cvaccuracies, df_model_info, df_ens_features, df_ens_test = run_models(clfs, df_train, df_test, train_idx, validation_idx, name_prefix, cv=True, ens=True)
    df_results_lr_cv = pd.concat([df_results_an_cv, df_model_cvaccuracies], axis=0)
    df_results_lr = pd.concat([df_results_an, df_model_info], axis=0)

Lasso Regression Selection
##################################


AttributeError: type object 'MyFeatureSelection' has no attribute 'LassoRegressionSelection'

In [113]:
lr_results_an

,0,1,2,3,4
lr_d_15/49MyIBL,0.681564,0.764045,0.769663,0.702247,0.797753
lr_d_15/49RandomForestClassifier,0.793296,0.775281,0.814607,0.786517,0.786517
lr_d_15/49SVC,0.837989,0.825843,0.820225,0.797753,0.876404
lr_d_15/49XGBClassifier,0.826816,0.825843,0.853933,0.792135,0.876404
lr_d_15/49MLPClassifier,0.826816,0.769663,0.825843,0.808989,0.853933
lr_d_15/49KNeighborsClassifier,0.798883,0.797753,0.842697,0.803371,0.865169
lr_d_15/49LinearDiscriminantAnalysis,0.798883,0.814607,0.808989,0.797753,0.853933


In [114]:
lr_results

,Accuracy,Duration
lr_d_15/49MyIBL,0.743054,2.634204
lr_d_15/49RandomForestClassifier,0.791243,0.024600
lr_d_15/49SVC,0.831643,0.031588
lr_d_15/49XGBClassifier,0.835026,0.143847
lr_d_15/49MLPClassifier,0.817049,4.326701
lr_d_15/49KNeighborsClassifier,0.821574,0.014692
lr_d_15/49LinearDiscriminantAnalysis,0.814833,0.019175
